In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
from sklearn.manifold import TSNE

In [3]:
user_prod_data = pd.read_csv('/kaggle/input/data-for-clst/user_prod_data.csv')

In [4]:
user_prod_data.product_id.nunique()

30217

In [5]:
user_prod_data_clstr = user_prod_data[['user_id', 'product_id']]
user_prod_data_clstr['here'] = 1

tmp = user_prod_data_clstr.groupby('product_id').product_id.count().to_frame('prc').reset_index()
imp_prod = set(tmp[tmp.prc >= 1000].product_id.unique())



/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [6]:
imp_prod_short = set(tmp[tmp.prc >= 5000].product_id.unique())
len(imp_prod_short)

42

In [7]:
 user_prod_data_clstr_imp =  user_prod_data_clstr[user_prod_data_clstr.product_id.apply(lambda s: s in imp_prod)]

In [8]:
dat = user_prod_data_clstr_imp.set_index(['user_id', 'product_id']).unstack().reset_index().fillna(0)

In [9]:
users = dat.values[:, 0]

In [10]:
X = dat.values[:, 1:]


In [11]:
del dat 

In [12]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=10, n_iter=7, random_state=42)
X_svd = svd.fit_transform(X)

In [13]:
user_data = pd.DataFrame(data = X_svd, columns = list(range(10)))
user_data['user_id'] = users

In [14]:
user_data.head()

,0,1,2,3,4,5,6,7,8,9,user_id
0,0.677227,-0.566403,0.359971,-0.316184,0.900356,-0.022820,-0.180347,-0.067487,-0.188025,0.163086,51.0
1,0.541329,-0.369569,-0.096065,0.046897,0.241228,-0.272353,0.091007,0.606664,-0.381499,-0.368358,65.0
2,0.707674,-0.698710,0.150276,-0.130393,0.586414,-0.244325,-0.273563,0.080410,-0.100727,0.083356,400.0
3,1.008989,-0.282310,-0.115016,0.437580,0.013321,-0.018396,-0.039588,0.329393,0.446118,0.453494,576.0
4,0.553552,-0.106798,-0.052603,0.225275,-0.291200,-0.034440,-0.206135,0.458004,0.193989,0.469732,608.0


In [15]:
user_prod_data.columns

Index(['user_id', 'product_id', 'master_category_id', 'parent_category_id',
       'order_id_nuni', 'quantity_median', 'price', 'discount', 'prod_type'],
      dtype='object')

In [16]:
prods_data = user_prod_data[['product_id', 'master_category_id', 'parent_category_id','price', 'discount', 'prod_type']]



In [17]:
prods_data_mean = prods_data.groupby(['product_id', 'master_category_id', 'parent_category_id', 'prod_type'])['price', 'discount'].mean().reset_index()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


In [18]:
prods_data_mean.shape, prods_data_mean.drop_duplicates().shape, prods_data_mean.product_id.nunique()

((30217, 6), (30217, 6), 30217)

In [19]:
prods_data_mean.head()

,product_id,master_category_id,parent_category_id,prod_type,price,discount
0,85,31.0,20,0,194.665361,2.427089
1,86,31.0,20,0,171.215447,5.021292
2,92,31.0,20,0,129.359207,24.875609
3,97,31.0,20,0,269.447982,1.439366
4,99,31.0,20,0,199.715313,2.569779


In [20]:
target_1_data = user_prod_data[['user_id', 'product_id', 'master_category_id', 'parent_category_id', 'price', 'discount', 'prod_type']]

In [21]:
target_1_data = pd.merge(target_1_data, user_data, on='user_id' )

In [22]:
target_1_set = set(zip(target_1_data.user_id.values, target_1_data.product_id.values))

In [33]:
tmpp = pd.DataFrame(columns=['user_id'] + list(imp_prod_short))


In [35]:
tmpp['user_id'] = target_1_data.user_id.unique()

In [36]:
tmpp.set_index('user_id', inplace=True)

In [39]:
tmpp = tmpp.fillna(1)

In [42]:
tmpp = tmpp.stack().reset_index()

In [45]:
tmpp.rename(columns={'level_1':'product_id'}, inplace=True)

In [46]:
target_0_set = set(zip(tmpp.user_id.values, tmpp.product_id.values))

In [48]:
target_0_set = target_0_set.difference(target_1_set)

In [56]:
trgt0_arr = np.array(list(target_0_set))

In [63]:
target_0 = pd.DataFrame(trgt0_arr, columns=['user_id', 'product_id'])
target_0.shape

(3930626, 2)

In [64]:
target_0 = pd.merge(target_0, prods_data_mean, on='product_id')
target_0.shape

(3930626, 7)

In [65]:
target_0 = pd.merge(target_0, user_data, on='user_id')
target_0.shape

(3930626, 17)

In [67]:
target_0['target'] = 0


In [68]:
target_1_data['target'] = 1

In [78]:
target_data = pd.concat([target_1_data, target_0])

In [85]:
import random

In [91]:
first_50000 = set(random.sample(list(target_data.user_id.unique()), 50000))


In [92]:
# first_part = target_data[target_data.user_id.apply(lambda s: s in first_50000)]
# second_part = target_data[target_data.user_id.apply(lambda s: not s in first_50000)]

In [114]:
targetusers10000 = random.sample(list(target_data.user_id.unique()), 10000)
targetusers_1 = set(targetusers10000[:5000])
targetusers_2 = set(targetusers10000[5000:])

In [115]:
first_part = target_data[target_data.user_id.apply(lambda s: s in targetusers_1)]


In [116]:
second_part = target_data[target_data.user_id.apply(lambda s:  s in targetusers_2)]
third_part = target_data[target_data.user_id.apply(lambda s:  (not s in targetusers_2) & (not s in targetusers_1))]

In [117]:
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier()

In [122]:
RF2 = RandomForestClassifier()

In [118]:
first_X = first_part[[  'master_category_id',       'parent_category_id',              'price',           'discount',
                'prod_type',                    0,                    1,
                          2,                    3,                    4,
                          5,                    6,                    7,
                          8,                    9,          ]].values

second_X = second_part[[  'master_category_id',       'parent_category_id',              'price',           'discount',
                'prod_type',                    0,                    1,
                          2,                    3,                    4,
                          5,                    6,                    7,
                          8,                    9,          ]].values

third_X = third_part[[  'master_category_id',       'parent_category_id',              'price',           'discount',
                'prod_type',                    0,                    1,
                          2,                    3,                    4,
                          5,                    6,                    7,
                          8,                    9,          ]].values

y_first = first_part['target'].values

y_second = second_part['target'].values
y_third = third_part['target'].values

In [106]:
y_train.shape

(4022267,)

In [119]:
RF.fit(first_X, y_first)

RandomForestClassifier()

In [ ]:
# RF2.fit(_X, y_first)

In [120]:
Y_test_second = RF.predict_proba(second_X)[:, 1]

In [121]:
Y_test_third = RF.predict_proba(third_X)[:, 1]

In [123]:
Y_test_second

array([1.  , 1.  , 1.  , ..., 0.01, 0.04, 0.03])

In [126]:
second_part['predicted'] = Y_test_second

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [127]:
second_part.to_csv('/kaggle/working/second_part.csv', index=False)

In [128]:
third_part['predicted'] = Y_test_third

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [130]:
third_part_res=third_part[['user_id', 'product_id', 'predicted']]

In [136]:
third_part_res = third_part_res.sort_values(by=['user_id', 'predicted'], ascending=[True, False])

In [145]:
third_part_res['product_id_str'] = third_part_res.product_id.apply(lambda s: str(s) + ' ')

In [146]:
third_part_res_tmp = third_part_res.groupby('user_id').product_id_str.sum().reset_index()

In [150]:
third_part_res_tmp['product_id'] = third_part_res_tmp.product_id_str.apply(lambda s: s.split()[:50])

50

In [155]:
some_prod = third_part_res_tmp.product_id.values[0]
third_part_res_tmp.product_id = third_part_res_tmp.product_id.apply(lambda s: s + some_prod[:50-len(s)])

In [157]:
third_part_res_tmp['Predicted'] = third_part_res_tmp.product_id.apply(lambda s: ' '.join(prod for prod in s))

In [159]:
third_part_res_tmp['Id'] = third_part_res_tmp['user_id']

In [162]:
third_part_res_tmp = third_part_res_tmp[['Id', 'Predicted']]

In [161]:
dum = pd.read_csv('/kaggle/input/qweqweq/dummy_submission.csv')

In [163]:
st = set(third_part_res_tmp.Id.unique())

In [164]:
dum = dum[dum.Id.apply(lambda s: not s in st)]

In [166]:
wer = pd.concat([dum, third_part_res_tmp])

In [169]:
wer.to_csv('/kaggle/working/wer.csv', index=False)

NameError: name 'index' is not defined